# Perceptions of Crime in Gauteng

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
#import household data
data = pd.read_csv("../data/raw/VOCS2016-2017-HOUSEHOLD_F1.csv")
data.head()


,uqno,prov,Q21topiccrime,Q21bHowLong,Q2211Murder,Q2212StrRob,Q2213HomeRob,Q2214BussRob,Q2215Hijack,Q2216Assault,...,Q2012Who,Q2013Recovered,Q2014Secure,Q2015Measures,Q2015Awareness,Q2015Gun,Q2015Weapon,Q2015Other,metro_code,Unnamed: 674
0,160100720000002601,1,2,3,2,2,2,2,2,1,...,9,9,9,9,9,9,9,9,1,NaN
1,160100720000006301,1,2,2,2,2,2,2,2,2,...,9,9,9,9,9,9,9,9,1,NaN
2,160100720000010001,1,2,1,2,2,2,2,2,2,...,9,9,9,9,9,9,9,9,1,NaN
3,160101220000003001,1,2,3,2,2,2,2,2,1,...,9,9,9,9,9,9,9,9,1,NaN
4,160101220000006301,1,2,3,2,2,2,2,2,1,...,9,9,9,9,9,9,9,9,1,NaN


Check data for problems:

In [22]:
#get overview of data frame
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21095 entries, 0 to 21094
Columns: 675 entries, uqno to Unnamed: 674
dtypes: float64(1), int64(674)
memory usage: 108.6 MB


In [3]:
#get gauteng only
gauteng = data[data['prov'] == 7]
gauteng.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5243 entries, 11815 to 17057
Columns: 675 entries, uqno to Unnamed: 674
dtypes: float64(1), int64(674)
memory usage: 27.0 MB


In [30]:
#get metros: check - is it the right metro codes? right number of respondents by metro?
metro = gauteng.groupby('metro_code') .size()
metro

metro_code
12     867
13    1389
14    1847
15    1140
dtype: int64

These metro codes correspond to those in the StatsSA documentation. Next, are there missing values?

In [32]:
# Function to calculate missing values by column
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [55]:
#check for missing values in data
missing_values_table(gauteng)

Your selected dataframe has 675 columns.
There are 1 columns that have missing values.


,Missing Values,% of Total Values
Unnamed: 674,5243,100.0


In [64]:
#remove unnamed column
gauteng = gauteng.drop(labels='Unnamed: 674', axis=1)

This Unnamed column contains no data, we can drop it from the data frame later. Alternatively, you can also find missing values in the following way:

In [65]:
#different way to check missing values
mis_val = (gauteng.isnull()
           .sum() 
           .reset_index() 
           .rename(columns={'index': 'Variable', 0: 'Missingfreq'}) 
           .sort_values('Missingfreq', ascending=False)
          .head())

mis_val

,Variable,Missingfreq
0,uqno,0
452,Q16a2b3Metal,0
444,Q15163CarryGun,0
445,Q15164CaryWeapon,0
446,Q15165Other,0


There are no missing values in the data (ignoring 'unnamed')

## What do people perceive as the most frequently occuring crimes in their area?

Get crime perceptions only

In [4]:
#get perceptions only
mostcrime = gauteng.iloc[:,4:27]
mostcrime.head()

,Q2211Murder,Q2212StrRob,Q2213HomeRob,Q2214BussRob,Q2215Hijack,Q2216Assault,Q2217SexAssult,Q2218ChildAbuse,Q2219Political,Q22110Vigilante,...,Q22114Burglary,Q22115Crop,Q22116Livestock,Q22117WhiteCollar,Q22118damage,Q22119IDtheft,Q22120HumanTrafficking,Q22121DrugTrafficking,Q22122HateCrime,Q22123Other
11815,1,1,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,1
11816,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
11817,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
11818,2,1,2,2,2,2,1,2,2,2,...,1,2,2,2,2,2,2,2,2,2
11819,2,2,1,1,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2


In [2]:
#Get responses into workable format:
#for every column:
#    sum yes (1) responses
#    divide sum of yes responses by lenth of column to get percentage of people who said that the crime occured freq

#Get most frequently occuring crimes:    
#get table of percentage and sort descending order    

#Get 5 most frequently occuring crimes and make bar graph of percentages

Is this what you would expect? Repeat for each metro - do the top 5 crimes differ be metro? 

In [3]:
# can you wrap the commands above in a function to repeat for each metro?

# What are the most feared crimes in Gauteng?
Does this differ by metro?